In [5]:
#импортируем требуемые пакеты
import pandas as pd
import requests
import numpy as np
import json
from selenium import webdriver
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.webdriver.common.keys import Keys

In [6]:
text_file = open('ключи.txt')
api_keys = text_file.readlines()
api_keys = list(map(lambda x: x.rstrip('\n'),api_keys))
text_file.close()

In [3]:
browser = webdriver.Chrome()
browser.implicitly_wait(40)

KeyboardInterrupt: 

In [ ]:
i = 0 #номер ключа

## Скачивание json файлов

In [ ]:
#for dolgota in np.arange (19,23.5,0.5): #Калининград
#    for shirota in np.arange (54,55.6,0.5): #Калининград
#for dolgota in np.arange (27,180.5,0.5): #Восточное полушарие
#    for shirota in np.arange (41,81.5,0.5):#Восточное полушарие
#for dolgota in np.arange (-180,-169.5,0.5): #Западное полушарие
#    for shirota in np.arange (41,81.5,0.5): #Западное полушарие
        browser.get ('https://search-maps.yandex.ru/v1/?apikey=4d7ca15d-3c65-4264-82c7-bb47c5353fa1&text=сбербанк отделения&lang=ru_RU&results=1000&ll='+str(dolgota)+'%2C'+str(shirota)+'&spn=0.5%2C0.5')
        a = browser.find_elements_by_xpath('//body/pre')[0].text
        if '"statusCode":403,"error"' in a:
            print('новый ключ')
            i+=1
            continue
            
        else:
            f = open('output2\\'+str(dolgota)+'_'+str(shirota)+'.txt','w')
            try:
                f.write(a)
            except UnicodeEncodeError:
                continue
            f.close()

## Сбор данных из файлов

In [8]:
import os

In [9]:
df = pd.DataFrame(columns=['address','hours','phones','url','adr2'])
for file in os.listdir('output2'):
    f = open('output2\\'+file)
    file_size = os.path.getsize('output2\\'+file)
    if file_size ==0:
        f.close()
        continue
    else:
        d = json.load(f)
    f.close()
    num=d['properties']['ResponseMetaData']['SearchResponse']['found']
    for i in range (0,num):
        try:
            adr = d['features'][i]['properties']['CompanyMetaData']['address']
        except IndexError:
            print('конец файла')
            continue
        try:
            hrs = d['features'][i]['properties']['CompanyMetaData']['Hours']['text']
        except:
            hrs = 'нет режима работы'
            print('нет режима',file)
            continue
        try:
            phn=''
            for tel in range(0,len(d['features'][i]['properties']['CompanyMetaData']['Phones'])):
                phn=phn+' '+d['features'][i]['properties']['CompanyMetaData']['Phones'][tel]['formatted']
        except:
            phn= 'нет телефона'
            print('нет телефона',file)
            continue            
        try:
            url = d['features'][i]['properties']['CompanyMetaData']['url']
        except:
            url = 'нет сайта'
            print('нет сайта', file)
            continue
        ad2 = d['features'][i]['properties']['description']
        q = pd.DataFrame(data=[[adr,hrs,phn,url,ad2]],columns=['address','hours','phones','url','adr2'])
        df=df.append(q, ignore_index=True)
    print(file)
df.to_excel('данные Яндекс.xlsx')    

JSONDecodeError: Expecting value: line 1 column 1 (char 0)